In [ ]:
import json
import requests
import pandas as pd


apikey = 'FXTzPCGAVnKR8b7X8hwD4DCPWzfRFQOicrOENjU6'

cf_columns = "food protein carbohydrates fat saturatedFat transFat calories fiber calcium iron magnesium potassium sodium cholesterol sugar ".split()

nutID_df=pd.DataFrame(columns='nutrient_name ID#'.split())
nutID_df=nutID_df.append({'nutrient_name':'protein', 'ID#':'203'}, ignore_index=True)
nutID_df=nutID_df.append({'nutrient_name':'carbohydrate', 'ID#':'205'}, ignore_index=True)
nutID_df=nutID_df.append({'nutrient_name':'fat', 'ID#':'204'}, ignore_index=True)
nutID_df=nutID_df.append({'nutrient_name':'saturatedFat', 'ID#':'606'}, ignore_index=True)
nutID_df=nutID_df.append({'nutrient_name':'transFat', 'ID#':'605'}, ignore_index=True)
nutID_df=nutID_df.append({'nutrient_name':'calories', 'ID#':'208'}, ignore_index=True)
nutID_df=nutID_df.append({'nutrient_name':'fiber', 'ID#':'291'}, ignore_index=True)
nutID_df=nutID_df.append({'nutrient_name':'calcium', 'ID#':'301'}, ignore_index=True)
nutID_df=nutID_df.append({'nutrient_name':'iron', 'ID#':'303'}, ignore_index=True)
nutID_df=nutID_df.append({'nutrient_name':'magnesium', 'ID#':'304'}, ignore_index=True)
nutID_df=nutID_df.append({'nutrient_name':'potassium', 'ID#':'306'}, ignore_index=True)
nutID_df=nutID_df.append({'nutrient_name':'sodium', 'ID#':'307'}, ignore_index=True)
nutID_df=nutID_df.append({'nutrient_name':'cholesterol', 'ID#':'601'}, ignore_index=True)
nutID_df=nutID_df.append({'nutrient_name':'sugar', 'ID#':'269'}, ignore_index=True)


def cf(cf_columns):
  df = pd.DataFrame(columns=cf_columns)
  
  return df

def get_query(query, apikey, debug=False): 
  url = f'https://api.nal.usda.gov/fdc/v1/foods/search?api_key={apikey}&query={query}'
  r = requests.get(url)
  y_json = r.json()
  x_df = pd.json_normalize(y_json)
  b_series = x_df.loc[0]['foods']
  a_df = pd.DataFrame(b_series)
  query= a_df.loc[0]['fdcId']
  if debug:
    pass

  return query

def get_usda_data(query, apikey, debug=False):
  foodID = get_query(query, apikey, debug)
  r = requests.get('https://api.nal.usda.gov/fdc/v1/' + str(foodID) + '?api_key='+ apikey)
  y = r.json()
  x = pd.json_normalize(y)
  usda_df = pd.DataFrame(x)
  
  if debug:
    pass
  
  return usda_df

def usda_to_cf(query, apikey, debug=False): 
  a_df = get_usda_data(query, apikey, debug)
  food_char_list = []
  food=a_df.loc[0]['description']
  food_char_list.append(food)
  b_series = a_df.loc[0]
  usda_df = pd.DataFrame(b_series)
  cf_df = cf(cf_columns)
  num_cf_columns=cf_df.columns.tolist()
  amount=0
  for i in range(len(num_cf_columns)):
    index='labelNutrients.'+ str(cf_columns[i])+'.value'
    if i==0:
      continue
    else:
      try:
        amount=usda_df.loc[index][0]
        food_char_list.append(amount)
      except KeyError:
        amount=0
        food_char_list.append(amount)
  if food_char_list[6]==0:
    food_char_list=[]
    food_char_list.append(food)
    for k in range(len(nutID_df)):
      added=False
      for l in range(len(a_df.loc[0]['foodNutrients'])):  
        q_nutID=a_df.loc[0]['foodNutrients'][l]['nutrient']['number']              
        if q_nutID==str(nutID_df.loc[k]['ID#']):
            amount=a_df.loc[0]['foodNutrients'][l]['amount']
            if amount != 0.0:
              food_char_list.append(amount)
              added=True
              break
        elif l==len(a_df.loc[0]['foodNutrients'])-1 and added==False:
          amount=None
          food_char_list.append(amount)        

  return food_char_list


def append_to_cf(query):
  df = cf(cf_columns)
  ab = pd.Series(usda_to_cf(query,apikey), index = df.columns)
  df=df.append(ab,ignore_index=True)
  df=df.fillna(0)
  return df

def run_usda_api(query):
  df = append_to_cf(query) 
  display(df)

run_usda_api('baked potato')# insert food you want information about